In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


train_df = pd.read_csv('answer_train.csv')
lb = LabelEncoder()
train_df['label'] = lb.fit_transform(train_df['label'])
test_df = pd.read_csv('submit_example_test1.csv')
test_df.drop('label', axis=1, inplace=True)
df = pd.concat([train_df, test_df], axis=0)
df['file'] = df['filename'].apply(lambda x: x.split('.')[0][6:].replace('_', ''))

In [4]:
import jieba
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

cw = lambda x: list(jieba.cut(x))
df['words'] = df['file'].apply(cw)
tokenizer=Tokenizer() 
tokenizer.fit_on_texts(df['words'])
vocab=tokenizer.word_index

In [ ]:
X = df[~df['label'].isnull()]
y_train = X['label']
X_train = X['words']
X_test = df[df['label'].isnull()]['words']

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape, BatchNormalization
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D, Conv1D, MaxPooling1D
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.utils.np_utils import to_categorical
from keras import initializers
from keras import backend as K
from keras.engine.topology import Layer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import numpy as np
import tensorflow as tf

# 设置随机数种子
seed = 2020
np.random.seed(seed)
tf.set_random_seed(seed)

def NN():
    model = Sequential()
    model.add(Embedding(len(vocab) + 1, 300, input_length=30)) #使用Embeeding层将每个词编码转换为词向量
    model.add(Conv1D(256, 5, padding='same'))
    model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Conv1D(128, 5, padding='same'))
    model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Conv1D(64, 3, padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(BatchNormalization())  
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(20, activation='softmax'))   
    return model

In [ ]:
oof = np.zeros(len(X_train))
predictions = np.zeros((len(X_test), 20))
KF = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for fold_, (trn_idx, val_idx) in enumerate(KF.split(X_train.values, y_train.values)):
    print("fold n°{}".format(fold_))
    print('trn_idx:',trn_idx)
    print('val_idx:',val_idx)
    X_tr = X_train.iloc[trn_idx]
    X_val = X_train.iloc[val_idx]
    X_train_word_ids = tokenizer.texts_to_sequences(X_tr)
    X_valid_word_ids = tokenizer.texts_to_sequences(X_val)
    X_test_word_ids = tokenizer.texts_to_sequences(X_test)
    X_train_padded_seqs=pad_sequences(X_train_word_ids,maxlen=30)
    X_valid_padded_seqs=pad_sequences(X_valid_word_ids,maxlen=30)
    X_test_padded_seqs=pad_sequences(X_test_word_ids, maxlen=30)
    y_tr = y_train.iloc[trn_idx]
    y_tr = keras.utils.to_categorical(y_tr, num_classes=20)
    y_val = y_train.iloc[val_idx]
    y_val = keras.utils.to_categorical(y_val, num_classes=20)
    model = NN()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])      
    model.fit(X_train_padded_seqs, y_tr, epochs=50, batch_size=1024, validation_data=(X_valid_padded_seqs, y_val), callbacks=[ES])
    oof[val_idx] = model.predict_classes(X_valid_padded_seqs)
    temp = model.predict_proba(X_test_padded_seqs)
    predictions[:] += temp

In [ ]:
test_label = lb.inverse_transform([np.argmax(i) for i in predictions / np.array(5)])
test_df['label'] = test_label
test_df.to_csv('baseline_sub.csv', index=False)